In [72]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [71]:
df_sample = pd.read_excel("../data/sample_fully_annotated.xlsx")
df_sample

,Unnamed: 0.1,Unnamed: 0,DE,SDG,AB,AI,robotics,IOT,big_data,computing_infrastructure,...,Term4,Term5,Annotator,manual_sent,roberta_sent,vader_sent_compound,vader_sent,trans_sent,SentiBigNomics,senti_sent
0,0,0,Robotic surgery; thoracic; outcomes; learning-...,3,The number of thoracic surgery cases performed...,False,True,False,False,False,...,NaN,NaN,Claudia,Pos,Pos,0.9799,Pos,Neu,0.000000,Neu
1,1,1,Deep learning; Convolutional neural network; C...,3,The COVID-19 pandemic is an emerging respirato...,True,False,False,False,False,...,"('Neural Network', '0.03', '0.38', '0.59')",NaN,Claudia,Pos,Pos,0.9076,Pos,Neu,0.191056,Neu
2,2,2,Sustainable cities; COVID-19 pandemic; video s...,3,Sustainable smart city initiatives around the ...,True,False,False,False,False,...,NaN,NaN,Claudia,Pos,Pos,0.9912,Pos,Neu,0.379250,Pos
3,3,3,technology; education; application; augmented ...,3,The use of technology in education is increasi...,True,False,False,False,False,...,NaN,NaN,Claudia,Pos,Neu,0.6486,Pos,Neu,0.000000,Neu
4,4,4,Artificial intelligence; Machine learning; Dee...,3,Colorectal cancer (CRC) was the second-ranked ...,True,False,False,False,False,...,"('Machine learning', '0.05', '0.23', '0.73')",NaN,Claudia,Pos,Pos,-0.7717,Neg,Neu,0.391800,Pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,135,135,global climate model; regional climate model; ...,13,Demonstrating the effect that climate change i...,False,False,False,False,True,...,NaN,NaN,Kevin,Pos,Pos,0.8591,Pos,Neu,0.000000,Neu
136,136,136,Lustre file system; I/O performance evaluation...,13,In this paper we study the performance of the ...,False,False,False,False,True,...,NaN,NaN,Kevin,Neu,Neu,0.9477,Pos,Neu,0.194118,Neu
137,137,137,landscape metrics; forest management; artifici...,13,Plant diversity is a core value of forests and...,True,False,False,False,False,...,NaN,NaN,Kevin,Neu,Pos,0.8100,Pos,Neu,0.000000,Neu
138,138,138,data exploration; data visualization; energy p...,13,The energy performance certificate (EPC) is a ...,True,False,False,False,False,...,NaN,NaN,Kevin,Pos,Pos,0.9887,Pos,Neu,0.000000,Neu


In [64]:
# Vader stuff

import numpy as np

analyzer = SentimentIntensityAnalyzer()
# for sentence in df_sample.AB:
#     vs = analyzer.polarity_scores(sentence)
#     # print("{:-<65} {}".format(sentence, str(vs)))
#     print(vs)
df_sample['vader_sent_compound'] = df_sample['AB'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
dic_vader = {}
df_sample['vader_sent'] = np.where(df_sample['vader_sent_compound'] > 0.6, 'Pos', (np.where(df_sample['vader_sent_compound'] > -0.6, 'Neu', 'Neg')))

In [81]:
def compute_accuracy_recall_f1(sent, manual_sent, model_sent):
    size = len(manual_sent)
    tot_correct_annot = 0
    tot_annot = 0
    tot_manual = 0
    for i in range(size):
        if manual_sent[i] == sent:
            tot_manual += 1
            if model_sent[i] == sent:
                tot_correct_annot += 1
        if model_sent[i] == sent:
            tot_annot += 1

    precision = tot_correct_annot / tot_annot
    recall = tot_correct_annot / tot_manual
    fscore = 2 * precision * recall / (precision + recall) if (precision+recall) != 0 else 0

    return sent, round(precision, 2), round(recall,2 ), round(fscore, 2), tot_correct_annot, tot_manual, tot_annot

def compute_acc(manual_sent, model_sent):
    tot_correct = 0
    size = len(manual_sent)
    for i in range(size):
        if manual_sent[i] == model_sent[i]:
            tot_correct += 1
    return round(tot_correct/size, 2)



In [19]:
"""
# Other sent analysis

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline
from tqdm import tqdm

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                           tokenizer=sentiment_model_path)

dic_sent = {"neutral": "Neu", "positive": "Pos", "negative":'Neg'}
lst_label = []
lst_score = []
for ind, row in df_sample.iterrows():
    txt = row.AB[-2000:]
    sent = sentiment_model([txt])[0]
    label = sent['label']
    score = sent['score']
    lst_label.append(dic_sent[label])
    lst_score.append(score)
df_sample['trans_sent'] = lst_label
df_sample.to_excel("../data/sample_fully_annotated.xlsx")
"""

In [82]:
lst_manual = df_sample.manual_sent.to_list()
dic_tool = {
"vader" : df_sample.vader_sent.to_list(),
"roberta" : df_sample.roberta_sent.to_list(),
"base" : df_sample.trans_sent.to_list(),
"senti" : df_sample.senti_sent.to_list(),
}

lst = []

for tool in dic_tool:
    for sent in ['Neg', 'Neu', 'Pos']:
        res = [tool]+list(compute_accuracy_recall_f1(sent=sent, manual_sent=lst_manual, model_sent=dic_tool[tool])) + [(compute_acc(manual_sent=lst_manual, model_sent=dic_tool[tool]))]
        lst.append(res)


In [93]:
df_res = pd.DataFrame(data=lst, columns=['Tool', 'Sentiment', "Precision", 'Recall', 'F1score', 'total_correct', 'total_manual', 'total_annotated', 'overall_accuracy'])
df_res.pivot(index="Tool", columns='Sentiment')

Precision             Recall             F1score              \
Sentiment       Neg   Neu   Pos    Neg   Neu   Pos     Neg   Neu   Pos   
Tool                                                                     
base           0.40  0.37  0.00   0.50  0.96  0.00    0.44  0.54  0.00   
roberta        0.33  0.60  0.76   0.50  0.54  0.79    0.40  0.57  0.77   
senti          0.00  0.41  0.77   0.00  0.87  0.24    0.00  0.55  0.36   
vader          0.08  0.32  0.62   0.25  0.13  0.77    0.12  0.19  0.69   

          total_correct  ...     total_manual         total_annotated       \
Sentiment           Neg  ... Pos          Neg Neu Pos             Neg  Neu   
Tool                     ...                                                 
base                  2  ...   0            4  52  84               5  134   
roberta               2  ...  66            4  52  84               6   47   
senti                 0  ...  20            4  52  84               3  111   
vader                 1  ...  65            4  52  84              13   22   

               overall_accuracy              
Sentiment  Pos              Neg   Neu   Pos  
Tool                                         
base         1             0.37  0.37  0.37  
roberta     87             0.69  0.69  0.69  
senti       26             0.46  0.46  0.46  
vader      105             0.52  0.52  0.52  

[4 rows x 21 columns]

In [95]:
df_res.set_index(['Tool', 'Sentiment']).to_excel("../results/score.xlsx")